# First linear regression model
This notebook will illustrate the contents of the lecture and tutorial in the week 5.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

## Load the rent index data
The data stems from a rent index in Munich from 1999 and stems from the [Textbook of L. Fahrmeir et al.](https://link.springer.com/book/10.1007/978-3-642-01837-4).

In [ ]:
mietspiegel = pd.read_table("data/mietspiegel99.raw")
print(mietspiegel.dtypes)
mietspiegel.head(5)
print(mietspiegel.shape)

| Variable  | Description                       | Categories |
|-----------|-----------------------------------|------------|
|miete      |Net rent per month (in DM)         | *numeric* |
|mieteqm    |Net rent per square meter (in DM)  | *numeric* |
|flaeche    |Living area in square meter        | *numeric* |
|bjahr      |Year of construction (in years)    | *numeric* |
|lage       |Exposure of the dwelling            |1 = norm |  
|           |                                    |2 = g  d |
|           |                                    |3 = excellent |
|bad        |Furnishings in the bathroom         |0 = normal |
|           |                                    |1 = upper |
|kueche     |Furnishings in the kitchen         | 0 = normal |
|           |                                    |1 = upper |
|zh         |Central heating                     |0 = no |
|           |                                    |1 = yes |
|bez        |District in Munich                | *numeric* |
ich	(numeric)


## Some data preparations

In [ ]:
mietspiegel.bjahr = mietspiegel.bjahr.astype(int)
#mietspiegel = mietspiegel[mietspiegel.bjahr <= 1995]
mietspiegel

## Descriptive analyses
We compute descriptive statistics for all variables and visualize the correlation of all variables

In [ ]:
mietspiegel.describe()

In [ ]:
colnames = mietspiegel.columns.values
#print(colnames)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(mietspiegel.corr())
fig.colorbar(cax)

#set the column names as axis tick labels
xaxis = np.arange(len(colnames))
ax.set_xticks(xaxis)
ax.set_yticks(xaxis)
ax.set_xticklabels(colnames)
ax.set_yticklabels(colnames)

plt.show()

## First regression model
The rent (`miete`) has a high correlation with the living space area (`flaeche`).

In [ ]:
mietspiegel.plot.scatter(x="flaeche", y="miete")

In [ ]:
# Create linear regression object
regr1 = linear_model.LinearRegression()

#extract the variables that we need from the dataset
y = mietspiegel.miete.values
x = mietspiegel.flaeche.values

# they currently have the shape (3059,) but we need (3059,1), see https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.fit
print(y.shape)
len = x.size
x = x.reshape(len, 1)
y = y.reshape(len, 1)
print(y.shape)

# We fit a linear regression model
regr1.fit(x, y)

# The regression coefficient
print("Coefficients: \n", regr1.coef_)
print("Intercept: \n", regr1.intercept_)


## Interpretation of the results
The model estimates an intercept / constant of 262.45, which means that a (hypothetical) appartment with 0m² living area has a base price of 262.48 DM. The coefficient of 9.40 means that for every 1m² 9.40 DM are added to the price.

We plot the estimated regression line in the scatterplot. The line has the slope of 9.40 and an its intersection with the y-axis at 262.45.

In [ ]:
pred = regr1.predict(x)
plt.scatter(x, y, color="black")
plt.plot(x, pred, color="red")
plt.show()

# Linear regression using the statsmodel library
We see that the interface to the regression model with scikit learn is very limited. This is not surprising, given that scikit learn is a library primarily used for machine learning application. If more insights are needed into regression models -- indeed, linear regression is a very powerful tool to understand data and conduct exploratory data analysis -- you can use the statsmodel library. 

Further learning resources using the statsmodel library with python can be found in an [online book by the Simon Fraser University](https://www.sfu.ca/~mjbrydon/tutorials/BAinPy/09_regression.html).

In [ ]:
x2 = sm.add_constant(x)
regr2 = sm.OLS(y, x2)
regr2_result = regr2.fit()
regr2_result.summary()

## Analysis of residuals

One assumption of the linear regression is that the residuals are normally distributed (it is called *homoscedasticity*, or homogeneity of variances). To check this, we can plot the distributions. 

### Exercise: Visually analyzing the residuals
1. Use a histogram to plot the residuals (hint: you can access them using `regr2_result.resid`.
2. Use a boxplot to visualize the residuals. Are they normally distributed?
3. Now, use a quantile-quantile (QQ-plot) to compare the distribution of the residuals with the normal distribution. 

In [ ]:
# 1. using the histogram


# 2. using the boxplot


# 3. using a QQ plot


## Multinomial linear regression

We want to further improve the model and add further independent variables. We start with a dummy `glage` variable encoding a better exposure of the object.

### Exercise: Include dummy variables
1. Prepare a dummy variable for exposure
2. Fit the regression and visualize the regression lines in the scatterplot
   

In [ ]:
#please implement here


In [ ]:
# visualize the fitted regression lines in the scatterplot


# Use a dummy variable with three categories
For more categories, we can use the build-in function of pandas to do the encoding.

### Expercise: Multiple levels in the dummy variable
1. After defining the new dummy variable with three levels, use it in the regression mode
2. Plot the three lines in the scatterplot

In [ ]:
#x3 = mietspiegel[["flaeche", "lage"]]
dlage = pd.get_dummies(mietspiegel.lage, # the variable to be encoded
                       prefix="dlage",   #prefix of the name of the dummy variables
                       drop_first=True,  #remove the first value of the dummy to avoid perfect colinearity
                       dtype=float)      #dummy should be numeric, not boolean
print(dlage.head(5))

In [ ]:
#use the new dummy in the regression


In [ ]:
#visualize the new estimates in the scatterplot
